In [ ]:
import sys
sys.path.append('../code/')

In [ ]:
import numpy as np
import pickle
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from labs import (get_terms, 
                  get_depth_optimized_terms, 
                  get_gate_optimized_terms_naive, 
                  get_gate_optimized_terms_greedy)
from pytket.extensions.quantinuum import QuantinuumBackend
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.backends.resulthandle import ResultHandle
from qaoa_qiskit import get_qaoa_circuit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from labs import negative_merit_factor_from_bitstring, true_optimal_mf
from utils import precompute_energies, transpile_hseries
from pytket.extensions.qiskit import AerStateBackend, AerBackend
from scipy import stats

In [ ]:
with open('../qokit/assets/best_known_QAOA_parameters_wrt_MF.json', 'r') as f:
    params = json.load(f)

params = map(lambda y: (str(y['N'])+'_'+str(y['p']), {'params': (y['beta'], y['gamma']), 'ar': y['AR'], 'merit_factor' : y['merit factor']}), filter(lambda x: x['N'] <= 20 and x['p'] <= 3, params.values()))
params = dict(params)

In [ ]:
def compute_energy_distribution(N, counts):
    num_shots = sum(counts.values())
     
    _distr = [(v, -1*negative_merit_factor_from_bitstring(np.array(list(map(lambda t: int(t), list(s)))))) for s, v in counts.items()]
    
    expanded_counts = sum([v[0]*[v[1]] for v in _distr], [])
    
    bins = np.bincount(((N**2)/(2*np.array(sorted(expanded_counts)))).astype(int))
    return bins[bins>0]/num_shots

def compute_mean_merit_factor(N, counts):
    num_shots = sum(counts.values())
    return sum([v * -1*negative_merit_factor_from_bitstring(np.array(list(map(lambda t: int(t), list(s))))) for s, v in counts.items()])/num_shots

def compute_merit_factor_ar(N, counts):
    worst_val = -1*max(precompute_energies(negative_merit_factor_from_bitstring, N))
    samp_mean = compute_mean_merit_factor(N, counts)
    return abs(samp_mean - worst_val)/abs((worst_val - true_optimal_mf[N]))

def compute_merit_conf_interval(N, counts, alpha=0.99):
    """
    compute the alpha-confidence interval
    """
    _distr = [(v, -1*negative_merit_factor_from_bitstring(np.array(list(map(lambda t: int(t), list(s)))))) for s, v in counts.items()]
    
    expanded_counts = sum([v[0]*[v[1]] for v in _distr], [])

    return stats.bootstrap((expanded_counts, ), np.mean, confidence_level=alpha)

def compute_p_value(N, counts):
    """
    compute probability of observing obtained sample mean under uniform distribution
    """
    num_shots = sum(counts.values()) 
    uni_mean_merit, uni_stddev, _ = uniform_stats(N)
    sample_mean = compute_mean_merit_factor(N, counts)
    k = np.abs(sample_mean - uni_mean_merit)//(uni_stddev/np.sqrt(num_shots))
    return 1/k**2

def uniform_stats(N):
    """
    Results from sampling bitstrings under uniform distribution
    """
    neg_merit_factors = precompute_energies(negative_merit_factor_from_bitstring, N)
    mean_merit = -1*np.mean(neg_merit_factors)
    stddev_merit = np.std(neg_merit_factors)
    
    bins = np.bincount(-((N**2)/(2*np.array(neg_merit_factors))).astype(int))
    energy_bins = bins[bins>0]/(2**N)
    
    return mean_merit,  stddev_merit, energy_bins

In [ ]:
def save_counts(counts, path):
    with open(path, 'wb') as f:
        pickle.dump(counts, f)
        
def result_from_file(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    

backend = QuantinuumBackend(device_name="H1-1E")  
def get_circuit_from_problem(N, p, terms_gen_func, to_qiskit=True):
    circuit, _  = build_circuit(backend, N, p, params[f'{N}_{p}']['params'], terms_gen_func=terms_gen_func)
    if to_qiskit:
        circuit = tk_to_qiskit(circuit, replace_implicit_swaps=True)
    return circuit
     
def run_sim(circuit, num_shots, err =0, save_res=False):
    dep = depolarizing_error(err, 2)
    noise_bit_flip = NoiseModel()
    noise_bit_flip.add_all_qubit_quantum_error(dep, ["cx", "rzz"])
    sim_noise = AerSimulator(noise_model=noise_bit_flip)
    results = sim_noise.run(circuit, shots=num_shots).result()
    counts = results.get_counts()
    if save_res:
        save_counts(counts,f'labs_hardware/local_sims/sim_n_{N}_er_{err}_counts.pkl')
    return counts

def run_sim_noisless(circuit, num_shots, save_res=False):
    sim_noise = AerSimulator()
    results = sim_noise.run(circuit, shots=num_shots).result()
    counts = results.get_counts()
    if save_res:
        save_counts(counts,f'labs_hardware/local_sims/sim_n_{N}_er_0_counts.pkl')
    return counts

In [ ]:
def display_uniform(N):
    mean_merit,  stddev_merit, energy_bins = uniform_stats(N)
    print(f'Uniform(mean_merit={mean_merit:0.3f})')
    plt.plot(energy_bins, label='Uniform')

    plt.xlabel('Energy Index')
    plt.ylabel('Probability')
    plt.title(f'Energies for N={N}, p=1')
    plt.legend()
    print(energy_bins[0])
    
    
def display_results_from_counts_file(file, legend_label):
    counts = result_from_file(file)
    
    N = len(list(counts.keys())[0])
    
    mean_merit_factor = compute_mean_merit_factor(N, counts)
    mean_merit_conf_int = compute_merit_conf_interval(N, counts)
    mean_merit_conf_int = mean_merit_conf_int.confidence_interval
    #ar = compute_merit_factor_ar(N, counts)
    p_value = compute_p_value(N, counts)
    energy_distr = compute_energy_distribution(N, counts)
    print(energy_distr[0])
    print(f'{legend_label}(mean_merit={mean_merit_factor:0.3f}, conf_int=[{mean_merit_conf_int.low:0.3f},{mean_merit_conf_int.high:0.3f}], p-value={p_value:0.3f})')
    
    plt.plot(energy_distr, label=legend_label)

    plt.xlabel('Energy Index')
    plt.ylabel('Probability')
    plt.title(f'Energies for N={N}, p=1')
    plt.legend()

In [ ]:
         
fig = plt.figure()
display_uniform(19)
display_results_from_counts_file('./labs_hardware/local_sims/sim_n_19_er_0_counts.pkl', 'Noiseless')
display_results_from_counts_file('./labs_hardware/device_runs/emul_n19_p1_greedy_opt_2171232717599320288_retry.pkl', 'Emulator')
plt.show()

<h2> Hardware Runs </h2>

In [ ]:
N = 18
num_shots = 2000
terms = get_gate_optimized_terms_greedy(N, seed=2171232717599320288, number_of_gate_zones=None)
beta, gamma = params[f'{N}_1']['params']
circuit = get_qaoa_circuit(N, terms, beta, gamma, save_statevector=False)
circuit.measure_all()

backend = QuantinuumBackend(device_name="H1-1E")
circuit, circuit_stats = transpile_hseries(backend, circuit)
print(circuit_stats)
circuit.name = f'qaoa_{N}_1'
# uncomment to push to backend
#backend.process_circuit(circuit, num_shots)

In [ ]:
## The below snippet fetches results from backend and saves to a pickle
handles_str = []
save_file_name = ''
# MAKE SURE N IS CORRECT WHEN RUNNING
N = 19
backend = QuantinuumBackend(device_name="H1-1E")
def fetch_handles(handles_str):
    all_counts = []
    for handle_str in handles_str:
        job = backend.get_result(ResultHandle.from_str(f'("{handle_str}", "null")'))
        counts = job.get_counts()
        counts = {''.join(map(lambda x: str(x), reversed(k))) : v for k, v in counts.items()}
        all_counts.append(counts)
    return all_counts
counts = fetch_handles(handles_str)
with open(f'./labs_hardware/device_runs/{save_file_name}.pkl' , 'wb') as f:
    pickle.dump(counts[0], f)